# Test if distortion coefficients are set and ordered

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import hyperspy.api as hs

import numpy as np
from distortions import Distortion

test_dict = {'A00' : 0, 'B11' : 0, 'A11' : 0, 'B22' : 0, 
             'A20' : 0, 'A22' : 0, 'B33' : 0, 'B31' : 0,
             'A31' : 0, 'A33' : 0}
# set some random values
keys = test_dict.keys()
values = np.random.random(len(keys)) + 1j*np.random.random(len(keys))
for key, value in zip(keys, values):
    test_dict[key] = value*0.00001
    
kdist = Distortion(**test_dict)
coeffsA = kdist._coeffs.copy()

# now scramble the dictionary
keys = list(test_dict.keys())
indices = np.arange(len(keys))
np.random.shuffle(indices)
new_dict = {}
for idx in indices:
    new_dict[keys[idx]] = test_dict[keys[idx]]

kdist = Distortion(**new_dict)
coeffsB = kdist._coeffs.copy()

assert coeffsA == coeffsB

# compare with input
test_dictA = {}
test_dictB = {}
for key in test_dict.keys():
    test_dictA[key] = coeffsA[key][3]
    test_dictB[key] = coeffsB[key][3]
    
assert test_dict == test_dictA
assert test_dict == test_dictB

/home/aeljarrat/anaconda3/envs/hyperspy_dev/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
kdist.save_coefficients('prueba.txt')

In [3]:
kdist.load_coefficients('prueba.txt')

In [4]:
kdist().plot()

In [5]:
from inline_holo import ModifiedImage as MI
from scipy.misc import ascent

img = MI(ascent())
img_1 = kdist(img)

hs.plot.plot_images([img, img_1], per_row=3)

In [6]:
from inline_holo import ModifiedImage as MI
from scipy.misc import ascent

img = MI(ascent())

kdist = Distortion(A00=10.+10j)
img_1 = kdist(img)

kdist = Distortion(A00=-10.-10j)
img_2 = kdist(img_1)

hs.plot.plot_images([img, img_1, img_2], per_row=3)

In [7]:
from inline_holo import ModifiedImage as MI
from scipy.misc import ascent
from scipy.ndimage import gaussian_filter

#img = MI(ascent())
imdata = np.kron([[1, 0] * 4, [0, 1] * 4] * 4, np.ones((25, 25)))
#imdata = gaussian_filter(imdata, 0.5)
img = MI(imdata)

value = -0.1

kdist = Distortion(A11=value, B11=-value)
img_1 = kdist(img, order=1)

ivalue = value/np.sqrt(np.abs(value)*2.)
kdist = Distortion(A11=-ivalue, B11=ivalue)
img_2 = kdist(img_1, order=1)

axs = hs.plot.plot_images([img, img_1, img_2], per_row=3)

for ax in axs[1:]:
    axs[0].get_shared_x_axes().join(axs[0], ax)
    axs[0].get_shared_y_axes().join(axs[0], ax)

In [76]:
from scipy.ndimage import map_coordinates
map_coordinates?

In [13]:
from inline_holo import ModifiedImage as MI
from scipy.misc import ascent

#kdist = Krivanek(A11=1, B11=-1, A22=-0.001-0.001j, B22=-0.001+0.001j)
#kdist = Krivanek(A00=10+10j, A11=0.001)

alpha = np.pi / 4
phase = np.exp(1j*alpha)
kdist = Krivanek(A11=phase, B11=-phase)

img = MI(ascent())
img_f = kdist(img)

# invert
alpha = -np.pi / 4
phase = np.exp(1j*alpha)
kdist = Krivanek(A11=phase, B11=-phase)

img_i = kdist(img_f)

hs.plot.plot_images([img, img_f, img_i])

In [9]:
img_f.plot()

In [16]:
a = 5+4j
print("%.4e" % a.real)

5.0000e+00


In [9]:
stack = []
kdist = Krivanek(A00=-1.-1j)
stack += kdist()
kdist = Krivanek(B11=-1.-1j)
stack += kdist()
kdist = Krivanek(A11=-1.-1j)
stack += kdist()
kdist = Krivanek(B22=-1.-1j)
stack += kdist()
kdist = Krivanek(A20=-1.-1j)
stack += kdist()
kdist = Krivanek(A22=-1.-1j)
stack += kdist()

stackr = [img.real for img in stack]
hs.plot.plot_images(stackr, no_nans=True)

stacki = [img.imag for img in stack]
hs.plot.plot_images(stacki, no_nans=True)

In [24]:
phi

array([[-2.3561945 , -2.3581553 , -2.3601236 , ...,  2.3601236 ,
         2.3581553 ,  2.3561945 ],
       [-2.3542337 , -2.3561945 , -2.3581629 , ...,  2.3581629 ,
         2.3561945 ,  2.3542337 ],
       [-2.3522651 , -2.3542259 , -2.3561945 , ...,  2.3561945 ,
         2.3542259 ,  2.3522651 ],
       ...,
       [-0.78932744, -0.7873667 , -0.7853982 , ...,  0.7853982 ,
         0.7873667 ,  0.78932744],
       [-0.78735894, -0.7853982 , -0.7834297 , ...,  0.7834297 ,
         0.7853982 ,  0.78735894],
       [-0.7853982 , -0.7834374 , -0.7814689 , ...,  0.7814689 ,
         0.7834374 ,  0.7853982 ]], dtype=float32)

In [27]:
nmp = []
for key in coeffs.keys():
    value = coeffs[key]
    nmp += [ value[:-1], ]
    Cp   = value[-1] 
nmp = np.array(nmp)
odd = nmp[:, 1] < 0


array([ True, False,  True, False,  True,  True, False, False,  True,
        True])